## Resampling (using kd_tree.resample_nearest)

In [4]:
import numpy as np
import xarray as xr
import h5py
import rasterio
import os

from rasterio.io import DatasetWriter
from rasterio.transform import from_origin
from pyresample import kd_tree, geometry, create_area_def
import matplotlib.pyplot as plt
import tempfile

In [5]:
name = '2A.GPM.Ku_38_pipeline_test_3'

filename = f'/home/aniket/aniket-backup/anthromet/imerg-data/ku/{name}.HDF5'

ka = f"/home/aniket/aniket-backup/anthromet/imerg-data/ka/2A.GPM.Ka.V9-20211125.20181011-S071532-E084806.026239.V07A.hdf5"
ku = "/home/aniket/aniket-backup/anthromet/imerg-data/ku/2A.GPM.Ku.V9-20211125.20230101-S000208-E013440.050238.V07A.hdf5"
dpr = "/home/aniket/Downloads/2B.GPM.DPRGMI.CORRA2022.20230101-S000208-E013440.050238.V07A.hdf5"

In [6]:
band_variables = [
    "/FS/SLV/precipRateNearSurface",
    # "/FS/SLV/precipRate",
    "/FS/PRE/elevation",
    '/FS/CSF/nHeavyIcePrecip',
]

In [7]:
def descend_obj(obj,sep='\t'):
    """
    Iterate through groups in a HDF5 file and prints the groups and datasets names and datasets attributes
    """
    if type(obj) in [h5py._hl.group.Group,h5py._hl.files.File]:
        for key in obj.keys():
            print(sep,'-',key,':',obj[key])
            descend_obj(obj[key],sep=sep+'\t')
    elif type(obj)==h5py._hl.dataset.Dataset:
        for key in obj.attrs.keys():
            print(sep+'\t','-',key,':',obj.attrs[key])

In [11]:
data = h5py.File(dpr, 'r')

# longitude = data['/FS/Longitude'][:]
# latitude = data['/FS/Latitude'][:]

resolution = 0.1



In [21]:
def get_all_dataset_from_group(group_name, file):
    group = file[group_name]
    paths = []
    for item in group.items():
        if(type(item[1])==h5py._hl.dataset.Dataset):
            paths.append(f"{group_name}/{item[0]}")
    return paths

In [22]:
get_all_dataset_from_group("KuGMI", data)

['KuGMI/Latitude',
 'KuGMI/Longitude',
 'KuGMI/sunLocalTime',
 'KuGMI/surfaceAirPressure',
 'KuGMI/surfaceAirTemperature',
 'KuGMI/surfaceVaporDensity',
 'KuGMI/skinTemperature',
 'KuGMI/skinTempSigma',
 'KuGMI/envParamNode',
 'KuGMI/airPressure',
 'KuGMI/airTemperature',
 'KuGMI/vaporDensity',
 'KuGMI/cloudLiqWaterCont',
 'KuGMI/cloudIceWaterCont',
 'KuGMI/lowestUnclutteredBin',
 'KuGMI/lowestEstimateBin',
 'KuGMI/phaseBinNodes',
 'KuGMI/precipTotDm',
 'KuGMI/precipTotLogNw',
 'KuGMI/precipTotMu',
 'KuGMI/precipTotWaterCont',
 'KuGMI/precipTotWaterContSigma',
 'KuGMI/precipLiqWaterCont',
 'KuGMI/precipTotRate',
 'KuGMI/precipTotRateSigma',
 'KuGMI/precipLiqRate',
 'KuGMI/multiScatMaxContrib',
 'KuGMI/nubfPIAfactor',
 'KuGMI/nearSurfPrecipTotRate',
 'KuGMI/nearSurfPrecipTotRateSigma',
 'KuGMI/nearSurfPrecipLiqRate',
 'KuGMI/estimSurfPrecipTotRate',
 'KuGMI/estimSurfPrecipTotRateSigma',
 'KuGMI/estimSurfPrecipLiqRate',
 'KuGMI/tenMeterWindSpeed',
 'KuGMI/tenMeterWindSigma',
 'KuGMI/surf

In [9]:
def discover_dataset_paths(hdf5_file, parent_path="/"):
    paths = []
    
    def visit_func(name):
        if isinstance(hdf5_file[name], h5py.Dataset):
            paths.append(name)
            
    hdf5_file.visit(visit_func)
    
    return paths


In [10]:
dataset_paths = discover_dataset_paths(data)

# Print the dataset paths
for path in dataset_paths:
    print(path)

AlgorithmRuntimeInfo
FS/CSF/binBBBottom
FS/CSF/binBBPeak
FS/CSF/binBBTop
FS/CSF/binHeavyIcePrecipBottom
FS/CSF/binHeavyIcePrecipTop
FS/CSF/flagBB
FS/CSF/flagHeavyIcePrecip
FS/CSF/flagShallowRain
FS/CSF/heightBB
FS/CSF/nHeavyIcePrecip
FS/CSF/qualityBB
FS/CSF/qualityTypePrecip
FS/CSF/typePrecip
FS/CSF/widthBB
FS/DSD/binNode
FS/DSD/paramRDm
FS/DSD/phase
FS/Experimental/precipRateESurface2
FS/Experimental/precipRateESurface2Status
FS/Experimental/seaIceConcentration
FS/Experimental/sigmaZeroProfile
FS/FLG/flagEcho
FS/FLG/flagScanPattern
FS/FLG/flagSensor
FS/FLG/qualityData
FS/FLG/qualityFlag
FS/Latitude
FS/Longitude
FS/PRE/adjustFactor
FS/PRE/binClutterFreeBottom
FS/PRE/binMirrorImageL2
FS/PRE/binRealSurface
FS/PRE/binStormTop
FS/PRE/echoCountRealSurface
FS/PRE/elevation
FS/PRE/ellipsoidBinOffset
FS/PRE/flagPrecip
FS/PRE/flagSigmaZeroSaturation
FS/PRE/height
FS/PRE/heightStormTop
FS/PRE/landSurfaceType
FS/PRE/localZenithAngle
FS/PRE/sigmaZeroMeasured
FS/PRE/snRatioAtRealSurface
FS/PRE/snow

In [11]:
attributes = data.attrs.get('FileHeader')

attributes = attributes.tobytes().decode().split(";\n")

attribute_dict = {}

for att in attributes:
    if "=" in att:
        key, val = att.split("=")
        attribute_dict[key] = val

attribute_dict['start_time'] = attribute_dict['StartGranuleDateTime']
attribute_dict['end_time'] = attribute_dict['StopGranuleDateTime']
attribute_dict

{'DOI': '10.5067/GPM/DPR/Ka/2A/07',
 'DOIauthority': 'http://dx.doi.org/',
 'DOIshortName': '2AKa',
 'AlgorithmID': '2AKa',
 'AlgorithmVersion': '9.20211125',
 'FileName': '2A.GPM.Ka.V9-20211125.20181011-S071532-E084806.026239.V07A.HDF5',
 'SatelliteName': 'GPM',
 'InstrumentName': 'DPR',
 'GenerationDateTime': '2022-01-28T15:53:29.000Z',
 'StartGranuleDateTime': '2018-10-11T07:15:32.781Z',
 'StopGranuleDateTime': '2018-10-11T08:48:07.815Z',
 'GranuleNumber': '26239',
 'NumberOfSwaths': '2',
 'NumberOfGrids': '0',
 'GranuleStart': 'SOUTHERNMOST_LATITUDE',
 'TimeInterval': 'ORBIT',
 'ProcessingSystem': 'PPS',
 'ProductVersion': 'V07A',
 'EmptyGranule': 'NOT_EMPTY',
 'MissingData': '0',
 'start_time': '2018-10-11T07:15:32.781Z',
 'end_time': '2018-10-11T08:48:07.815Z'}

In [12]:
data_dict = {} 

for var_name in band_variables:
    var_data = data[var_name]
    if var_data.ndim == 2:
        print(f"adding data for {var_name}")
        data_dict[var_name] = var_data[:]
    elif var_data.ndim == 3:
        last_dim = var_data.shape[-1]
        dimension_name = str(var_data.attrs.get('DimensionNames')).replace("'", "").split(',')[-1]
        for i in range(last_dim):
            new_var_name = f"{var_name}_{dimension_name}_{i}"
            print(f"adding data for {new_var_name}")
            data_dict[new_var_name] = var_data[:,:,i]
    else:
        print(f"data of dimension {var_data.ndim} not handled")

adding data for /FS/SLV/precipRateNearSurface
adding data for /FS/PRE/elevation
adding data for /FS/CSF/nHeavyIcePrecip


In [13]:
area_def = create_area_def('world',
                           "+proj=longlat +datum=WGS84 +no_defs +type=crs", # crs
                           area_extent=[-180, -70, 180, 70],
                           resolution=resolution)

swath_def = geometry.SwathDefinition(lons=longitude, lats=latitude)

crs = rasterio.crs.CRS.from_epsg(4326)

#upper left corner west, north and pixel sizes xsize, ysize.
transform = from_origin(-180, 70, resolution, resolution) 


In [16]:
var_data = data["/FS/PRE/elevation"][:]
resampled_data = kd_tree.resample_nearest(swath_def, var_data, area_def, radius_of_influence=5000, epsilon=0, fill_value=np.nan)

resampled_data.shape

(1400, 3600)

In [19]:
from osgeo import gdal, osr

output_file = f'gdal_generated_1.tif'
with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = os.path.join(temp_dir, output_file)

    dataset = gdal.GetDriverByName('MEM').Create(
        '',
        1400, # height
        3600, # width
        1, # one band
        gdal.GDT_Float32,
        options=['TILED=YES']
    )

    # dataset.SetGeoTransform(transform)
    # Make the spatial reference (just WGS84, lat/lon).
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # https://epsg.io/4326
    dataset.SetProjection(srs.ExportToWkt())

    band = dataset.GetRasterBand(1)

    var_data = data["/FS/PRE/elevation"][:]
    resampled_data = kd_tree.resample_nearest(swath_def, var_data, area_def, radius_of_influence=5000, epsilon=0, fill_value=np.nan)

    band.SetDescription("Elevation")
    band.WriteArray(resampled_data)

    dataset.FlushCache()

    dataset2
    dataset2 = gdal.GetDriverByName('GTiff').CreateCopy(
        temp_path,
        dataset,
        options=['COPY_SRC_OVERVIEWS=YES', 'TILED=YES', 'COMPRESS=LZW'],
    )
    dataset2.FlushCache()

    dataset2 = None
    dataset = None


Warning 6: driver MEM does not support creation option TILED


ValueError: array larger than output file, or offset off edge

In [11]:
output_file = f'{name}_check.tif'
with rasterio.open(output_file, 'w',
                   driver='GTiff',
                   height=1400,
                   width=3600,
                   count=len(data_dict),
                   dtype=np.float32, 
                   crs=crs, 
                   transform=transform, nodata=np.nan, compress='lzw') as dst:
    for band, (var_name, var_data) in enumerate(data_dict.items(), start=1):
        print(f"resampling band {var_name}")
        var_data = var_data.astype(np.float32)
        resampled_data = kd_tree.resample_nearest(swath_def, var_data, area_def, radius_of_influence=5000, epsilon=0, fill_value=np.nan)
        
        print(f"writing band {var_name} to {output_file}")
        dst.write(resampled_data, band)

        dst.set_band_description(band, var_name)
        dst.update_tags(band, band_name=var_name)

        dst.update_tags(**attribute_dict)


resampling band /FS/SLV/precipRateNearSurface
writing band /FS/SLV/precipRateNearSurface to 2A.GPM.Ku_38_pipeline_test_3_check.tif
resampling band /FS/PRE/elevation
writing band /FS/PRE/elevation to 2A.GPM.Ku_38_pipeline_test_3_check.tif
resampling band /FS/CSF/nHeavyIcePrecip
writing band /FS/CSF/nHeavyIcePrecip to 2A.GPM.Ku_38_pipeline_test_3_check.tif


In [5]:
from osgeo import gdal
import numpy as np

x_size = 1000
y_size = 1000
num_bands = 1
file_name = "gdal_test.tiff"

data = np.ones((num_bands, y_size, x_size))
driver = gdal.GetDriverByName('GTiff')
data_set = driver.Create(file_name, x_size, y_size, num_bands,
gdal.GDT_Float32,
options=["TILED=YES",
"COMPRESS=LZW",
"INTERLEAVE=BAND"])

for i in range(num_bands):
    data_set.GetRasterBand(i + 1).WriteArray(data[i])
    data_set = None